# Example how to access remote HDF5 files 
-- at https://hsds-kc.ideaconsult.net


In [ ]:
from keycloak import KeycloakOpenID
import h5pyd 
class H5Wrapper:
    def __init__(self,user,password):
        self.tokens = H5Wrapper.get_token(user,password)

    @staticmethod
    def get_token(user,password):
        keycloak_openid = KeycloakOpenID(server_url="https://iam.ideaconsult.net/auth",
                                    client_id='idea-ui',
                                    realm_name='nano',
                                    client_secret_key='')

        token = keycloak_openid.token(user,password)
        return token

    def File(self,path,mode="r",retries=3):
        return h5pyd.File(path,mode=mode,retries=retries,api_key=self.tokens["access_token"])

    def Folder(self,path):
        return h5pyd.File(path,api_key=self.tokens["access_token"])            

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
user_widget = widgets.Text(
    value='',
    placeholder='',
    description='User:',
    disabled=False
)
pass_widget = widgets.Password(
    value='',
    placeholder='',
    description='Password:',
    disabled=False
)
display(user_widget,pass_widget)

In [ ]:
api_key = H5Wrapper.get_token(user_widget.value,pass_widget.value)
print(api_key["access_token"])

## Example with h5pyd

In [ ]:
import h5pyd
import matplotlib.pyplot as plt

# This is a path to a remote h5 file, not a local path
path = "/Round_Robin_1/FMNT-Madrid/BWTek iRaman/532/NeonSNQ043_iR532_Probe_stitched.cha"
# example how to read the remote HDF5 files (without ramanchada1 or ramanchada2 dependencies)
with h5pyd.File(path,api_key=api_key) as f:
    for key in f.keys():
        print(key, f[key])
        for a in f[key].attrs:
            print(key,"attribute",a,f[key].attrs[a])        
        if isinstance(f[key], h5pyd.Dataset):
            dset = f[key]
            print("Dataset",key,dset,dset.shape,dset.size,dset.ndim,dset.dtype)
            print(dset[...])
            print("x",dset[0])
            print("y",dset[1])
            plt.plot(dset[0],dset[1])
            plt.xlabel(dset.dims[0].label)
            plt.ylabel(dset.dims[1].label)
            sample = f["annotation_sample"].attrs["sample"]
            instrument = f["annotation_study"].attrs["instrument"]
            wavelength = f["annotation_study"].attrs["wavelength"]
            partner = f["annotation_study"].attrs["provider"]
            investigation = f["annotation_study"].attrs["investigation"]
            plt.suptitle("{} ({},{}nm) by {} [{}]".format(sample, instrument, wavelength, partner, investigation))




## Example with ramanchada1

## Example with ramanchda2

In [ ]:
from ramanchada2.io import HSDS

h5pyw = H5Wrapper(user_widget.value,pass_widget.value)
x, y, meta = HSDS.read_cha(path,dataset='/raw',h5module=h5pyw)
#this is metadata from the native file, not the curated metadata from annotation_x groups as above
print(meta)
plt.plot(x,y)

In [ ]:
from ramanchada2.io import HSDS

x, y, meta = HSDS.read_cha(path,dataset='/raw',h5module=h5pyw)
#this is metadata from the native file, not the curated metadata from annotation_x groups as above
print(meta)
plt.plot(x,y)

In [ ]:
# example creating Spectrum class , reading raw dataset from a remote HDF5 file created by ramanchada2
from ramanchada2.spectrum.creators.from_chada import from_chada

spe = from_chada(path,dataset="/raw",h5module=h5pyw)
spe.plot()



### Processing example

In [ ]:
ss0 = spe - spe.moving_minimum(80)
ss1 = spe - spe.moving_minimum(15)
fig, ax = plt.subplots(1, 2, figsize=(15, 5), tight_layout=True)
spe.plot(ax=ax[0], label='original')
ss0.plot(ax=ax[0], label='movmin(80)')
ss1.plot(ax=ax[0], label='movmin(15)')

#spe.plot(ax=ax[1], label='original')
ss0.plot(ax=ax[1], label='movmin(80)')
ss1.plot(ax=ax[1], label='movmin(15)')
ax[1].set_ylim(0, 8000)

In [ ]:
### normalize

In [ ]:
spe = spe.normalize()
spe.plot()